In [1]:
import os, warnings
import wandb

import pandas as pd
from fastai.vision.all import *
from sklearn.model_selection import StratifiedGroupKFold

import params
warnings.filterwarnings('ignore')

In [2]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="data_split")

wandb: Currently logged in as: darek. Use `wandb login --relogin` to force relogin


In [3]:
raw_data_at = run.use_artifact(f'{params.RAW_DATA_AT}:latest')
path = Path(raw_data_at.download())

wandb: Downloading large artifact bdd_simple_1k:latest, 813.75MB. 4007 files... Done. 0:0:2.5


In [4]:
path.ls()

(#5) [Path('artifacts/bdd_simple_1k:v0/eda_table.table.json'),Path('artifacts/bdd_simple_1k:v0/images'),Path('artifacts/bdd_simple_1k:v0/LICENSE.txt'),Path('artifacts/bdd_simple_1k:v0/labels'),Path('artifacts/bdd_simple_1k:v0/media')]

In [5]:
fnames = os.listdir(path/'images')
groups = [s.split('-')[0] for s in fnames]

In [6]:
orig_eda_table = raw_data_at.get("eda_table")

wandb: Downloading large artifact bdd_simple_1k:latest, 813.75MB. 4007 files... Done. 0:0:0.6


In [7]:
y = orig_eda_table.get_column('bicycle')

In [8]:
df = pd.DataFrame()
df['File_Name'] = fnames
df['fold'] = -1

In [9]:
cv = StratifiedGroupKFold(n_splits=10)
for i, (train_idxs, test_idxs) in enumerate(cv.split(fnames, y, groups)):
    df.loc[test_idxs, ['fold']] = i

In [10]:
df['Stage'] = 'train'
df.loc[df.fold == 0, ['Stage']] = 'test'
df.loc[df.fold == 1, ['Stage']] = 'valid'
del df['fold']
df.Stage.value_counts()

train    800
valid    100
test     100
Name: Stage, dtype: int64

In [11]:
df.to_csv('data_split.csv', index=False)

In [12]:
processed_data_at = wandb.Artifact(params.PROCESSED_DATA_AT, type="split_data")

In [13]:
processed_data_at.add_file('data_split.csv')
processed_data_at.add_dir(path)

wandb: Adding directory to artifact (./artifacts/bdd_simple_1k:v0)... Done. 1.7s


In [14]:
data_split_table = wandb.Table(dataframe=df[['File_Name', 'Stage']])

In [15]:
join_table = wandb.JoinedTable(orig_eda_table, data_split_table, "File_Name")

In [16]:
processed_data_at.add(join_table, "eda_table_data_split")

<ManifestEntry digest: ACjVT6MoG11Qdi/beHVJpQ==>

In [17]:
run.log_artifact(processed_data_at)
run.finish()